# Section 5.1 Basic Decorators

In [2]:
def pythonic_decorator(func):
    def wrapper(*args,**kwargs):
        print('Now Calling: ' + func.__name__)
        return func(*args,**kwargs)
    return wrapper

@pythonic_decorator
def addNums(a,b):
    return a+b

addNums(7,6)

Now Calling: addNums


13